In [1]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from delfi.simulator.Gauss import Gauss


n_params = 1
seed = 42
m = Gauss(dim=n_params, seed=seed, noise_cov=1.)
p = dd.Gaussian(m=0.*np.ones(n_params), 
                S=1.*np.eye(n_params),
                seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)

obs = 0.8 * np.ones((1,1))

def calc_posterior(obs,
                   uniform=False, 
                   n_summary=1,
                   prior_mu=0.*np.ones(n_params),
                   prior_cov=np.atleast_2d(1.*np.eye(n_params)),
                   noise_cov=np.atleast_2d(1.)):
    if not uniform:
        posterior_cov = np.linalg.inv(np.linalg.inv(prior_cov)+n_summary*np.linalg.inv(noise_cov))
        posterior_mu = np.dot(posterior_cov, (n_summary*np.dot(np.linalg.inv(noise_cov), obs) + \
                       np.dot(np.linalg.inv(prior_cov), prior_mu)))
        return dd.Gaussian(m=posterior_mu.reshape(-1), S=posterior_cov)
    else:
        posterior_mu = obs
        posterior_cov = noise_cov/n_summary
        return dd.Gaussian(m=posterior_mu, S=posterior_cov)

posterior = calc_posterior(obs)
print('Analytical solution')
print(posterior.m)
print(posterior.S)

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Analytical solution
[ 0.4]
[[ 0.5]]


In [2]:
# BASIC

res = infer.Basic(g, n_hiddens=[10], seed=seed, svi=False)
log, trn_data, posterior = res.run(10000, lr=0.0001, epochs=1000)

print('BASIC algo solution')
print(res.predict(obs).xs[0].m)
print(res.predict(obs).xs[0].S)


BASIC algo solution
[ 0.38509488]
[[ 0.49600515]]


In [4]:
# CDELFI

res = infer.CDELFI(g, obs=obs, n_hiddens=[10], seed=seed, svi=False)
log, trn_data, posterior = res.run(10000, n_rounds=2, lr=0.001, epochs=1000)
proposal = res.generator.proposal

print('CDELFI solution')
posterior = res.predict(obs)
print(posterior.xs[0].m)
print(posterior.xs[0].S)

print('CDELFI solution if the prior was the proposal prior (wrong solution)')
res.generator.proposal = None
posterior_uncorrected = res.predict(obs)
print(posterior_uncorrected.xs[0].m)
print(posterior_uncorrected.xs[0].S)


CDELFI solution
[ 0.43199023]
[[ 0.49047926]]
CDELFI solution if the prior was the proposal prior (wrong solution)
[ 0.53410733]
[[ 0.32224068]]


In [3]:
# SNPE
n_rounds = 2

res = infer.SNPE(g, 
                 obs=obs, 
                 n_hiddens=[10], 
                 seed=seed, 
                 svi=True)

logs, tds, posteriors = res.run(10000, 
                                n_rounds=n_rounds, 
                                epochs=1000, 
                                minibatch=200,
                                monitor=['loss.iws', 
                                         'loss.lprobs', 
                                         'loss.kl'])

for r in range(n_rounds):
    print('round {}'.format(r+1))
    print(posteriors[r].xs[0].m)
    print(posteriors[r].xs[0].S)
    
# .4 / .5 is the correct answer


round 1
[ 0.4219119]
[[ 0.50657564]]
round 2
[ 0.41620895]
[[ 0.4977428]]
